In [ ]:
import numpy as np 
import os
import math
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,Sequential,Model 
from keras.utils import np_utils
import matplotlib.pyplot as plt
from utils.ReadData import readData
from utils.SetGen import SetGen
from utils.DataAug import DataAugement

In [ ]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

gpus = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0],True)
print('Num GPUs Available:', len(tf.config.experimental.list_physical_devices('GPU')))

from tensorflow.python.client import device_lib
def get_available_gpus():
    local_divice_protos = device_lib.list_local_devices()
    # print(local_divice_protos)
    return [x for x in local_divice_protos if x.device_type == 'GPU']   
print(get_available_gpus())

In [ ]:
path_LB= 'PlantarPressureDataset/LBNew'
path_RB = 'PlantarPressureDataset/RBNew'
path_N = 'PlantarPressureDataset/NormalNew'

LBTotal, LBLeft, LBRight = readData().Extract(path = path_LB)
RBTotal, RBLeft, RBRight = readData().Extract(path = path_RB)
NTotal, NLeft, NRight = readData().Extract(path = path_N)

In [ ]:
Data = LBTotal + RBTotal + NTotal
Data = np.reshape(Data,(np.shape(Data)+(1,)))

Label = []
for i in range(len(LBTotal)):
    Label.append(0)
for i in range(len(RBTotal)):
    Label.append(1)
for i in range(len(NTotal)):
    Label.append(2)

In [ ]:
Xtr, ytr, Xval, yval = SetGen().GetSets(Data, Label, 0.9, 0.1)
H = 48
W = 34

In [ ]:
Gen = DataAugement(Xtr,ytr)
Xtr, ytr =Gen.dataAug(angle = 5,rate = 0.35)
Dim = np.size(Xtr,axis = 0)

In [ ]:
ytr= np_utils.to_categorical(ytr,3)
yval = np_utils.to_categorical(yval,3)

In [ ]:
model = keras.Sequential(name = 'NewConvLSTM')
bs = 32
hidden_feature = 128
ts = 60
h = 48
w = 34
channel = 1

input_data = keras.Input((ts,h,w,channel))
x = layers.BatchNormalization()(input_data)
fil = np.array([4,8,16])

for i in range(3):
    x = layers.Conv3D(filters = 2**(i+2), kernel_size = (1,3,3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)


f1 = layers.Conv3D(16, (1, 2, 2), padding='same' )(x)
f2 = layers.Conv3D(16, (1, 3, 3), padding='same' )(x)
f3 = layers.Conv3D(16, (1, 4, 4), padding='same' )(x)


newInput = tf.concat([f1,f2,f3,x],axis = -1)
PDC0 = layers.Conv3D(32,(1,1,1),padding = 'same' )(newInput)
RPDC = layers.ReLU()(PDC0)
BN_RPDC = layers.BatchNormalization()(RPDC)
Residual = PDC0+BN_RPDC
ConvLSTM0 = layers.ConvLSTM2D(16,kernel_size = 3, padding = 'same',return_sequences = 'True')(Residual)
MaxPool0 = layers.MaxPool3D(pool_size = (1,2,2))(ConvLSTM0)
BN0 = layers.BatchNormalization()(MaxPool0)
ConvLSTM1 = layers.ConvLSTM2D(32,kernel_size = 3, padding = 'same',return_sequences = 'True')(BN0)
MaxPool1 = layers.MaxPool3D(pool_size = (1,2,2))(ConvLSTM1)
BN1 = layers.BatchNormalization()(MaxPool1)
ConvLSTM2 = layers.ConvLSTM2D(128,kernel_size = 3, padding = 'same', dropout=0.1)(BN1)
BN2 = layers.BatchNormalization()(ConvLSTM2)
Maxpool1 = layers.MaxPool2D(pool_size = (2,2))(BN2)
Flatten1 = layers.Flatten()(Maxpool1)

FC1 = layers.Dense(1000,activation = 'relu')(Flatten1)
DP0 = layers.Dropout(0.1)(FC1)
FC2 = layers.Dense(128,activation = 'relu')(DP0)
DP1 = layers.Dropout(0.1)(FC2)
FC3 = layers.Dense(16,activation = 'relu')(FC2)
outPut = layers.Dense(3,activation = 'softmax')(FC3)

model  = keras.Model(inputs = input_data, outputs = outPut)
model.summary()

In [ ]:
model.compile(optimizer='adam', loss = 'categorical_crossentropy',metrics=['accuracy'])

In [ ]:
BS = 32
N = np.size(Xtr,axis = 0)
# callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15)
# with tf.device('/GPU:0'):
#     history = model.fit(Xtr,ytr,batch_size = BS, epochs= N//BS, validation_data = (Xval,yval),callbacks=[callback])
history = model.fit(Xtr,ytr,batch_size = BS, epochs= N//BS, validation_data = (Xval,yval))

In [ ]:
model.save('model/model1',save_format='h5')


In [ ]:
model.save_weights('weights/model1')

In [ ]:
pd.DataFrame(history.history).to_csv('history/training_log1.csv', index=False)

In [ ]:
# plt.title("Acc")
# plt.semilogy(history.history['accuracy'], label='Train acc')
# plt.semilogy(history.history['val_accuracy'], label='Validation acc')
# plt.xlabel('Epochs')
# plt.ylabel('accuracy')
# plt.legend()

In [ ]:
#  tf.keras.utils.plot_model(model, to_file='convLSTM_new.png', show_shapes=True, show_layer_names=True,rankdir='TB', dpi=900, expand_nested=True)